In [1]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# GRAPH SETTINGS
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'

# Data

In [6]:
# Directorios base
local = '/mnt/lustre/home/samuloza'
#local = 'C:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado'
base_dirs = {
    "Competitive": f"{local}/data/samuel_lozano/cooked/map_kitchen/competitive",
    "Cooperative": f"{local}/data/samuel_lozano/cooked/map_kitchen/cooperative"
}

output_path = f"{local}/data/samuel_lozano/cooked/map_kitchen/training_results.csv"

# Eliminar el archivo CSV si ya existe
if os.path.exists(output_path):
    os.remove(output_path)

In [7]:
all_dfs = []

# Patrón para capturar los coeficientes de recompensa
reward_pattern = re.compile(
    r"'([^']+)':\s*\(\s*([-\d\.eE+]+),\s*([-\d\.eE+]+)\)"
)

In [10]:
for game_type, base_dir in base_dirs.items():
    game_flag = 1 if "Cooperative" in game_type else 0
    for folder in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder)
        if not os.path.isdir(folder_path):
            continue

        date_time_str = folder.replace("Training_", "")
        config_path = os.path.join(folder_path, "config.txt")
        csv_path = os.path.join(folder_path, "training_stats.csv")

        if not (os.path.exists(config_path) and os.path.exists(csv_path)):
            continue

        with open(config_path, "r") as f:
            config_contents = f.read()
        match = reward_pattern.findall(config_contents)
        if not match:
            continue

        (name_1, alpha_1, beta_1), (name_2, alpha_2, beta_2) = match
        alpha_1, beta_1 = float(alpha_1), float(beta_1) 
        alpha_2, beta_2 = float(alpha_2), float(beta_2)

        lr_match = re.search(r"LR:\s*([0-9.eE+-]+)", config_contents)
        lr = float(lr_match.group(1)) 

        with open(csv_path, 'r') as f:
            lines = f.readlines()
        header = lines[0]
        filtered_lines = [header] + [line for line in lines[1:] if not line.startswith("episode,env")]

        from io import StringIO
        df = pd.read_csv(StringIO("".join(filtered_lines)))

        df.iloc[:, 0] = range(1, len(df) + 1)

        df.insert(0, "timestamp", date_time_str)
        df.insert(1, "game_type", game_flag)
        df.insert(2, "alpha_1", alpha_1)
        df.insert(3, "beta_1", beta_1)
        df.insert(4, "alpha_2", alpha_2)
        df.insert(5, "beta_2", beta_2)
        df.insert(6, "lr", lr)

        all_dfs.append(df)

In [11]:
# Concatenar todos los resultados
final_df = pd.concat(all_dfs, ignore_index=True)
final_df.to_csv(output_path, index=False)

# Visualization

In [12]:
local = '/mnt/lustre/home/samuloza'
#local = 'C:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado'
base_dirs = {
    "Competitive": f"{local}/data/samuel_lozano/cooked/map_kitchen/competitive",
    "Cooperative": f"{local}/data/samuel_lozano/cooked/map_kitchen/cooperative"
}

output_path = f"{local}/data/samuel_lozano/cooked/map_kitchen/training_results.csv"

# Leer el CSV especificando los tipos de datos
dtype_dict = {
    "timestamp": str,
    "game_type": int,
    "alpha_1": float,
    "beta_1": float,
    "alpha_2": float,
    "beta_2": float
}

df = pd.read_csv(output_path, dtype=dtype_dict, low_memory=False)
for col in df.columns[6:]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Crear una columna identificadora de combinación de coeficientes
df = df.sort_values(by=["alpha_1", "alpha_2"], ascending=[False, False])
df["attitude_key"] = df.apply(lambda row: f"{row['alpha_1']}_{row['beta_1']}_{row['alpha_2']}_{row['beta_2']}", axis=1)
df["pure_reward_total"] = df["pure_reward_ai_rl_1"] + df["pure_reward_ai_rl_2"]

In [14]:
# Filtrar todas las combinaciones únicas
unique_attitudes = df["attitude_key"].unique()
unique_lr = df["lr"].unique()
unique_game_type = df["game_type"].unique()

figures_dir = f"{local}/data/samuel_lozano/cooked/map_kitchen/figures/"
os.makedirs(figures_dir, exist_ok=True)

rewarded_metrics_1 = [
    "delivered_ai_rl_1",
    "cut_ai_rl_1",
    "salad_ai_rl_1",
]

rewarded_metrics_2 = [
    "delivered_ai_rl_2",
    "cut_ai_rl_2",
    "salad_ai_rl_2",
]

metrics_1 = [
    "floor_actions_ai_rl_1",
    "wall_actions_ai_rl_1",
    "counter_actions_ai_rl_1",
    "dispenser_actions_ai_rl_1",
    "cutting_board_actions_ai_rl_1",
    "delivery_actions_ai_rl_1"
]

metrics_2 = [
    "floor_actions_ai_rl_2",
    "wall_actions_ai_rl_2",
    "counter_actions_ai_rl_2",
    "dispenser_actions_ai_rl_2",
    "cutting_board_actions_ai_rl_2",
    "delivery_actions_ai_rl_2"
]

rewarded_metric_labels = {
    "delivered_ai_rl_1": "Delivered",
    "cut_ai_rl_1": "Cut",
    "salad_ai_rl_1": "Salad"
}

metric_labels = {
    "floor_actions_ai_rl_1": "Floor",
    "wall_actions_ai_rl_1": "Wall",
    "counter_actions_ai_rl_1": "Counter",
    "dispenser_actions_ai_rl_1": "Dispenser",
    "cutting_board_actions_ai_rl_1": "Cutting Board",
    "delivery_actions_ai_rl_1": "Delivery"
}

rewarded_metric_colors = {
    "delivered_ai_rl_1": "#27AE60",
    "cut_ai_rl_1": "#E74C3C",
    "salad_ai_rl_1": "#9B59B6"
}

metric_colors = {
    "floor_actions_ai_rl_1": "#27AE60",
    "wall_actions_ai_rl_1": "#E74C3C",
    "counter_actions_ai_rl_1": "#9B59B6",
    "dispenser_actions_ai_rl_1": "#A0522D",
    "cutting_board_actions_ai_rl_1": "#95A5A6",
    "delivery_actions_ai_rl_1": "#95A5A6"
}

## General visuals (not smoothed)

In [15]:
# Print Pure total reward vs epoch

for attitude in unique_attitudes:
    subset = df[df["attitude_key"] == attitude]

    plt.figure(figsize=(10, 6))
    
    for game_type in unique_game_type:
        game_type_filtered = subset[subset["game_type"] == game_type]
    
        # Filtrar por tasa de aprendizaje
        for lr in unique_lr:
            lr_filtered = game_type_filtered[game_type_filtered["lr"] == lr]
            grouped = lr_filtered.groupby("epoch")["pure_reward_total"].mean().reset_index()
            label = f"Game Type {game_type}, LR {lr}"
            plt.plot(grouped["epoch"], grouped["pure_reward_total"], label=label)
    
    # Añadir detalles
    plt.title(f"Pure Reward vs Epoch\nAttitude {attitude}")
    plt.xlabel("Epoch")
    #plt.xlim([0,100])
    plt.ylabel("Pure Reward Total")
    plt.legend()
    plt.tight_layout()
    
    sanitized_attitude = attitude.replace('.', 'p')
    filename = f"pure_reward_attitude_{sanitized_attitude}.png"
    filepath = os.path.join(figures_dir, filename)
    plt.savefig(filepath)
    plt.close()

In [16]:
# Print Each agent pure total reward vs epoch

for attitude in unique_attitudes:
    subset = df[df["attitude_key"] == attitude]

    for game_type in unique_game_type:
        game_type_filtered = subset[subset["game_type"] == game_type]
    
        # Crear la figura
        plt.figure(figsize=(10, 6))
    
        # Filtrar por tasa de aprendizaje
        for lr in unique_lr:    
            lr_filtered = game_type_filtered[game_type_filtered["lr"] == lr]
            grouped = lr_filtered.groupby("epoch")[["pure_reward_ai_rl_1", "pure_reward_ai_rl_2"]].mean().reset_index()
            label_0 = f"Agent 1, LR {lr}"
            label_1 = f"Agent 2, LR {lr}"
            plt.plot(grouped["epoch"], grouped["pure_reward_ai_rl_1"], label=label_0)
            plt.plot(grouped["epoch"], grouped["pure_reward_ai_rl_2"], label=label_1)
    
        # Añadir detalles
        plt.title(f"Pure Reward vs Epoch\nAttitude {attitude}, Game Type {game_type}")
        plt.xlabel("Epoch")
        #plt.xlim([0,100])
        plt.ylabel("Pure Reward Total")
        plt.legend()
        plt.tight_layout()
        
        sanitized_attitude = attitude.replace('.', 'p')
        filename = f"pure_reward_agents_g{game_type}_attitude_{sanitized_attitude}.png"
        filepath = os.path.join(figures_dir, filename)
        plt.savefig(filepath)
        plt.close()

In [21]:
# Print agent metrics vs epoch
for attitude in unique_attitudes:
    subset = df[df["attitude_key"] == attitude]

    att_parts = attitude.split('_')
    att0_title = f"{att_parts[0]}_{att_parts[1]}"
    att1_title = f"{att_parts[2]}_{att_parts[3]}"

    for game_type in [0, 1]:
        for lr in subset["lr"].unique():
            filtered_subset = subset[(subset["game_type"] == game_type) & (subset["lr"] == lr)]
    
            plt.figure(figsize=(12, 6))
            for metric in rewarded_metrics_1:
                grouped = filtered_subset.groupby(["epoch"])[metric].mean().reset_index()
                plt.plot(grouped["epoch"], grouped[metric], label=metric.replace("_", " ").title())
            plt.title(f"Metrics per Epoch - Game Type {game_type}, LR {lr}, Attitude {att0_title}")
            plt.xlabel("Epoch")
            plt.ylabel("Mean value")
            plt.legend()
            #plt.xlim([0, 100])
            plt.tight_layout()
            
            sanitized_attitude = attitude.replace('.', 'p')
            filename_1 = f"rewarded_metrics_agent1_g{game_type}_lr{str(lr).replace('.', 'p')}_attitude_{sanitized_attitude}.png"
            filepath_1 = os.path.join(figures_dir, filename_1)
            plt.savefig(filepath_1)
            plt.close()
    
            plt.figure(figsize=(12, 6))
            for metric in rewarded_metrics_2:
                grouped = filtered_subset.groupby(["epoch"])[metric].mean().reset_index()
                plt.plot(grouped["epoch"], grouped[metric], label=metric.replace("_", " ").title())
            plt.title(f"Metrics per Epoch - Game Type {game_type}, LR {lr}, Attitude {att1_title}")
            plt.xlabel("Epoch")
            plt.ylabel("Mean value")
            plt.legend()
            #plt.xlim([0, 100])
            plt.tight_layout()

            filename_2 = f"rewarded_metrics_agent2_g{game_type}_lr{str(lr).replace('.', 'p')}_attitude_{sanitized_attitude}.png"
            filepath_2 = os.path.join(figures_dir, filename_2)
            plt.savefig(filepath_2)
            plt.close()

# Smoothed

In [18]:
N = 15

smoothed_figures_dir = f"{local}/data/samuel_lozano/cooked/map_kitchen/figures/smoothed_{N}/"
os.makedirs(smoothed_figures_dir, exist_ok=True)

In [25]:
# Print Pure total reward vs epoch

plt.figure(figsize=(10, 6))

for game_type in unique_game_type:
    game_type_filtered_df = df[df["game_type"] == game_type]

    # Filtrar por tasa de aprendizaje
    for lr in unique_lr:
        lr_filtered_df = game_type_filtered_df[game_type_filtered_df["lr"] == lr]
        lr_filtered_df["epoch_block"] = (lr_filtered_df["epoch"] // N)
        block_means = lr_filtered_df.groupby("epoch_block")["pure_reward_total"].mean()
        middle_epochs = lr_filtered_df.groupby("epoch_block")["epoch"].median()

        if game_type == 0:
            label = f"Competitive"
            color = "red"
        else:
            label = f"Cooperative"
            color = "green"
        
        plt.plot(middle_epochs, block_means, label=label, color=color)

# Añadir detalles
plt.xlabel("Epochs", fontsize=20)
#plt.xlim([0,100])
plt.ylabel("Mean total reward", fontsize=20)
plt.legend(fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()

filename = f"pure_reward_smoothed_{N}.png"
filepath = os.path.join(smoothed_figures_dir, filename)
plt.savefig(filepath)
plt.close()

In [26]:
# Print Each agent pure total reward vs epoch

for game_type in unique_game_type:
    game_type_filtered = df[df["game_type"] == game_type]

    # Crear la figura
    plt.figure(figsize=(10, 6))

    # Filtrar por tasa de aprendizaje
    for lr in unique_lr:
        lr_filtered = game_type_filtered[game_type_filtered["lr"] == lr]
        lr_filtered["epoch_block"] = (lr_filtered["epoch"] // N)
        block_means_1 = lr_filtered.groupby("epoch_block")["pure_reward_ai_rl_1"].mean()
        block_means_2 = lr_filtered.groupby("epoch_block")["pure_reward_ai_rl_2"].mean()
        middle_epochs = lr_filtered.groupby("epoch_block")["epoch"].median()

        label_1 = f"Agent 1, LR {lr}"
        label_2 = f"Agent 2, LR {lr}"
        plt.plot(middle_epochs, block_means_1, label=label_1)
        plt.plot(middle_epochs, block_means_2, label=label_2)

    # Añadir detalles
    plt.title(f"Pure Reward vs Epoch\nGame Type {game_type}")
    plt.xlabel("Epoch")
    #plt.xlim([0,100])
    plt.ylabel("Pure Reward Total")
    plt.legend()
    plt.tight_layout()
    
    filename = f"pure_reward_agents_g{game_type}_smoothed_{N}.png"
    filepath = os.path.join(smoothed_figures_dir, filename)
    plt.savefig(filepath)
    plt.close()

In [ ]:
# Print agent rewarded metrics vs epoch

for game_type in unique_game_type:
    for lr in unique_lr:
        filtered_subset = df[(df["game_type"] == game_type) & (df["lr"] == lr)]
        filtered_subset["epoch_block"] = (filtered_subset["epoch"] // N)

        plt.figure(figsize=(12, 6))
        for metric in rewarded_metrics_1:
            block_means = filtered_subset.groupby("epoch_block")[[metric]].mean()
            middle_epochs = filtered_subset.groupby("epoch_block")["epoch"].median()
            plt.plot(middle_epochs, block_means, label=metric.replace("_", " ").title())
        plt.title(f"Metrics per Epoch - Game Type {game_type}, LR {lr}")
        plt.xlabel("Epoch")
        plt.ylabel("Mean value")
        plt.legend()
        #plt.xlim([0, 100])
        plt.tight_layout()
        
        filename_1 = f"rewarded_metrics_agent1_g{game_type}_lr{str(lr).replace('.', 'p')}_smoothed_{N}.png"
        filepath_1 = os.path.join(smoothed_figures_dir, filename_1)
        plt.savefig(filepath_1)
        plt.close()
    
        plt.figure(figsize=(12, 6))
        
        for metric in rewarded_metrics_2:
            block_means = filtered_subset.groupby("epoch_block")[[metric]].mean()
            middle_epochs = filtered_subset.groupby("epoch_block")["epoch"].median()
            plt.plot(middle_epochs, block_means, label=metric.replace("_", " ").title())
        plt.title(f"Metrics per Epoch - Game Type {game_type}, LR {lr}")
        plt.xlabel("Epoch")
        plt.ylabel("Mean value")
        plt.legend()
        #plt.xlim([0, 100])
        plt.tight_layout()

        filename_2 = f"rewarded_metrics_agent2_g{game_type}_lr{str(lr).replace('.', 'p')}_smoothed_{N}.png"
        filepath_2 = os.path.join(smoothed_figures_dir, filename_2)
        plt.savefig(filepath_2)
        plt.close()

/tmp/ipykernel_3982513/93969600.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_subset["epoch_block"] = (filtered_subset["epoch"] // N)
/tmp/ipykernel_3982513/93969600.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_subset["epoch_block"] = (filtered_subset["epoch"] // N)


In [29]:
# Print agent metrics vs epoch

for game_type in unique_game_type:
    for lr in unique_lr:
        filtered_subset = df[(df["game_type"] == game_type) & (df["lr"] == lr)]
        filtered_subset["epoch_block"] = (filtered_subset["epoch"] // N)

        plt.figure(figsize=(12, 6))
        for metric in metrics_1:
            block_means = filtered_subset.groupby("epoch_block")[[metric]].mean()
            middle_epochs = filtered_subset.groupby("epoch_block")["epoch"].median()
            plt.plot(middle_epochs, block_means, label=metric.replace("_", " ").title())
        plt.title(f"Metrics per Epoch - Game Type {game_type}, LR {lr}")
        plt.xlabel("Epoch")
        plt.ylabel("Mean value")
        plt.legend()
        #plt.xlim([0, 100])
        plt.tight_layout()
        
        filename_1 = f"metrics_agent1_g{game_type}_lr{str(lr).replace('.', 'p')}_smoothed_{N}.png"
        filepath_1 = os.path.join(smoothed_figures_dir, filename_1)
        plt.savefig(filepath_1)
        plt.close()
    
        plt.figure(figsize=(12, 6))
        
        for metric in metrics_2:
            block_means = filtered_subset.groupby("epoch_block")[[metric]].mean()
            middle_epochs = filtered_subset.groupby("epoch_block")["epoch"].median()
            plt.plot(middle_epochs, block_means, label=metric.replace("_", " ").title())
        plt.title(f"Metrics per Epoch - Game Type {game_type}, LR {lr}")
        plt.xlabel("Epoch")
        plt.ylabel("Mean value")
        plt.legend()
        #plt.xlim([0, 100])
        plt.tight_layout()

        filename_2 = f"metrics_agent2_g{game_type}_lr{str(lr).replace('.', 'p')}_smoothed_{N}.png"
        filepath_2 = os.path.join(smoothed_figures_dir, filename_2)
        plt.savefig(filepath_2)
        plt.close()

/tmp/ipykernel_3982513/2572494477.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_subset["epoch_block"] = (filtered_subset["epoch"] // N)
/tmp/ipykernel_3982513/2572494477.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_subset["epoch_block"] = (filtered_subset["epoch"] // N)


In [28]:
# Print Pure total reward vs epoch
for attitude in unique_attitudes:
    subset = df[df["attitude_key"] == attitude]

    plt.figure(figsize=(10, 6))
    
    for game_type in unique_game_type:
        game_type_filtered = subset[subset["game_type"] == game_type]
    
        # Filtrar por tasa de aprendizaje
        for lr in unique_lr:
            lr_filtered = game_type_filtered[game_type_filtered["lr"] == lr]
            lr_filtered["epoch_block"] = (lr_filtered["epoch"] // N)

            # Calcular la media de recompensa por bloque
            block_means = lr_filtered.groupby("epoch_block")["pure_reward_total"].mean()
            middle_epochs = lr_filtered.groupby("epoch_block")["epoch"].median()

            if game_type == 0:
                label = f"Competitive"
                color = "red"
            else:
                label = f"Cooperative"
                color = "orange"

            plt.plot(middle_epochs, block_means, label=label, color=color)
    
    # Añadir detalles
    plt.xlabel("Epochs", fontsize=20)
    #plt.xlim([0,100])
    if attitude == "0.707107_0.707107_0.707107_0.707107":
        plt.title("Cooperative agents", fontsize=24)
    elif attitude == "1.0_0.0_1.0_0.0":
        plt.title("Individualistic agents", fontsize=24)
    elif attitude == "-1.0_0.0_1.0_0.0":
        plt.title("Martyr agent with individualistic agent", fontsize=24)
    elif attitude == "-0.707107_-0.707107_0.707107_0.707107":
        plt.title("Destructive agent with cooperative agent", fontsize=24)

    plt.ylabel("Mean total reward", fontsize=20)
    plt.legend(fontsize=20)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    #plt.ylim(-140, 320)
    plt.tight_layout()
    
    sanitized_attitude = attitude.replace('.', 'p')
    filename = f"pure_reward_attitude_{sanitized_attitude}_smoothed_{N}.png"
    filepath = os.path.join(smoothed_figures_dir, filename)
    plt.savefig(filepath)
    plt.close()